In [1]:
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from scipy import stats
import pickle 
from pickle import load,dump
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import cudf
from lightgbm import LGBMClassifier, log_evaluation
import warnings
import joblib

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import random

In [2]:
oof_xgb = pd.read_csv('../input/fork-of-amex-finalxgb/oof_transfer_learning.csv')
oof_xgbpy = pd.read_csv('../input/amex-fe-pyramid/oof_xgb_v1.csv') 
oof_automl = pd.read_csv('../input/amex-lightautoml/AutoML_oof.csv')
oof_automl43 = pd.read_csv('../input/amex-lightautoml43/AutoML_oof.csv')
oof_cat = pd.read_csv('../input/amex-catboost770/oof_cat_v1.csv')
oof_cat41 = pd.read_csv('../input/amex-catboost770-seed41/oof_cat_seed41.csv')
oof_lgbm = pd.read_csv('../input/lgbm770/oof_lgbm_v1(1).csv')
oof_lgbmquick = pd.read_csv('../input/amexlightgbmquickstart2seed/oof_lgbmquick.csv')
oof_transfer1 = pd.read_csv('../input/amextransferlearning/oof_transfer_learning.csv')
oof_transfer2 = pd.read_csv('../input/amextransferlearningresnet/oof_transfer_learning.csv')
oof_cnn = pd.read_csv('../input/amex-keras-1d-cnn/oof_CNN1D.csv')

In [3]:
oof_cat['oof_pred_cat'] = (oof_cat['oof_pred_cat'] + oof_cat41['oof_pred_cat'])/2

In [4]:
oof_automl['oof_pred'] = (oof_automl['oof_pred'] + oof_automl43['oof_pred'])/2

In [5]:
oof_xgb.rename(columns={'oof_pred':'oof_xgb'},inplace=True)
oof_xgbpy.rename(columns={'oof_pred':'oof_xgbpy'},inplace=True)
oof_cat.rename(columns={'oof_pred_cat':'oof_cat'},inplace=True)
oof_automl.rename(columns={'oof_pred':'oof_automl'},inplace=True)
oof_transfer1.rename(columns={'oof_pred':'oof_transfer1'},inplace=True)
oof_transfer2.rename(columns={'oof_pred':'oof_transfer2'},inplace=True)
oof_cnn.rename(columns={'oof_pred':'oof_cnn'},inplace=True)
oof_lgbm.rename(columns={'oof_pred_lgbm':'oof_lgbm'},inplace=True)
oof_lgbmquick.rename(columns={'oof_pred':'oof_lgbmquick'},inplace=True)

In [6]:
oof_df = pd.concat([oof_xgb,oof_automl.oof_automl,oof_lgbm.oof_lgbm,oof_xgbpy.oof_xgbpy,oof_cat.oof_cat,oof_lgbmquick.oof_lgbmquick,oof_cnn.oof_cnn,oof_transfer1.oof_transfer1,oof_transfer2.oof_transfer2],axis=1)

In [7]:
oof_df.head(5)

,target,oof_xgb,oof_automl,oof_lgbm,oof_xgbpy,oof_cat,oof_lgbmquick,oof_cnn,oof_transfer1,oof_transfer2
0,1,1.154194,0.782662,1.056450,1.028873,0.708636,1.310415,0.611922,0.760796,0.664672
1,0,-7.939376,0.000712,0.000387,-8.107467,0.000534,-7.767016,0.049239,0.000401,0.000582
2,0,-6.199133,0.001576,0.001538,-6.588468,0.000739,-6.753305,0.069256,0.001334,0.001924
3,0,-3.873612,0.019122,0.022494,-3.595696,0.027202,-3.918158,0.184326,0.021304,0.030478
4,1,1.994368,0.918872,2.121106,2.096829,0.921224,1.792675,0.626901,0.852713,0.859151


scaler = StandardScaler()
scaler = load(open('../input/amex9predscaler/predscaler','rb'))
oofCols = [col for col in oof_df.columns if 'oof' in col]
oof_df.loc[:,oofCols] =  scaler.transform(oof_df.loc[:,oofCols])

In [8]:
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [9]:
def amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

def lgb_amex_metric(y_true, y_pred):
    """The competition metric with lightgbm's calling convention"""
    return ('amex',
            amex_metric(y_true, y_pred),
            True)

In [10]:
oof_lr = np.zeros((len(oof_df),))
oofCols = [col for col in oof_df.columns if 'oof' in col]
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
for fold,(train_ind,val_ind) in enumerate(skf.split(oof_df, oof_df.target)):
    tr_x, val_x = (
        oof_df[oofCols].iloc[train_ind].reset_index(drop=True),
        oof_df[oofCols].iloc[val_ind].reset_index(drop=True),
    )
    tr_y, val_y = (
        oof_df.target.iloc[train_ind].reset_index(drop=True),
        oof_df.target.iloc[val_ind].reset_index(drop=True),
    )
    
    final_estimator = LogisticRegression(penalty='l2',solver='liblinear',C=0.002,class_weight="balanced",max_iter=5000)
    final_estimator.fit(tr_x,tr_y)
    
    filename = f'lr_model{fold}.sav'
    pickle.dump(final_estimator, open(filename, 'wb'))
    
   # loaded_model = pickle.load(open('../input/amexlrmodel/lr_model.sav', 'rb'))
    
    preds = final_estimator.predict_proba(val_x)[:,1]
    
    acc = amex_metric_mod(val_y.values,preds)
    print(f'Kaggle Metric fold {fold} =',acc,'\n')
    
    oof_lr[val_ind] = preds
    
    del tr_x, val_x, tr_y, val_y,final_estimator
    _ = gc.collect()
    
print('#'*25)
acc = amex_metric_mod(oof_df.target,oof_lr)
print('OVERALL CV Kaggle Metric =',acc)    

Kaggle Metric fold 0 = 0.7970596133345098 

Kaggle Metric fold 1 = 0.7985678534334624 

Kaggle Metric fold 2 = 0.7989722380278486 

Kaggle Metric fold 3 = 0.7995280115358154 

Kaggle Metric fold 4 = 0.8006704022538105 

#########################
OVERALL CV Kaggle Metric = 0.7990021439369359


In [11]:
oof_lr = pd.DataFrame(oof_lr,columns=['oof_lr'])

In [12]:
oof_lr.to_csv('oof_lr.csv',index=False)

In [13]:
def my_booster(random_state=42, n_estimators=1300):
    return LGBMClassifier(n_estimators=n_estimators,
                          learning_rate=0.01, reg_lambda=300,
                          min_child_samples=1000,
                          num_leaves=50,
                          colsample_bytree= 0.1,
                          max_bins=511, random_state=random_state)

score_list = []
y_pred_list = []
oof_lgbm = np.zeros((len(oof_df)))
kf = StratifiedKFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(oof_df, oof_df.target)):
    X_tr, X_va, y_tr, y_va, model = None, None, None, None, None
    X_tr = oof_df.iloc[idx_tr][oofCols]
    X_va = oof_df.iloc[idx_va][oofCols]
    y_tr = oof_df.target[idx_tr]
    y_va = oof_df.target[idx_va]
    
    model = my_booster()
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning)
        model.fit(X_tr, y_tr,
                  eval_set = [(X_va, y_va)], 
                   eval_metric = 'AUC',
                  callbacks=[log_evaluation(100)])
    X_tr, y_tr = None, None
    y_va_pred = model.predict_proba(X_va, raw_score=True)
    score = amex_metric_mod(y_va, y_va_pred)
    
    joblib.dump(model, f'LGBM_stack_fold{fold}.pkl')
    
    # SAVE OOF
    oof_lgbm[idx_va] = y_va_pred
        
    n_trees = model.best_iteration_
    if n_trees is None: n_trees = model.n_estimators
    print(f"                Score = {score:.5f}")
    score_list.append(score)
    
print('#'*25)
val_score = amex_metric(oof_df.target, oof_lgbm)
print(f"Amex metric: {val_score}")    

[100]	valid_0's auc: 0.961947	valid_0's binary_logloss: 0.300459
[200]	valid_0's auc: 0.962374	valid_0's binary_logloss: 0.239256
[300]	valid_0's auc: 0.962872	valid_0's binary_logloss: 0.22179
[400]	valid_0's auc: 0.963002	valid_0's binary_logloss: 0.216508
[500]	valid_0's auc: 0.963063	valid_0's binary_logloss: 0.214783
[600]	valid_0's auc: 0.963062	valid_0's binary_logloss: 0.214164
[700]	valid_0's auc: 0.963074	valid_0's binary_logloss: 0.213912
[800]	valid_0's auc: 0.963072	valid_0's binary_logloss: 0.21381
[900]	valid_0's auc: 0.963062	valid_0's binary_logloss: 0.213767
[1000]	valid_0's auc: 0.963046	valid_0's binary_logloss: 0.213753
[1100]	valid_0's auc: 0.963034	valid_0's binary_logloss: 0.213753
[1200]	valid_0's auc: 0.963019	valid_0's binary_logloss: 0.213767
[1300]	valid_0's auc: 0.963004	valid_0's binary_logloss: 0.213784
                Score = 0.79876
[100]	valid_0's auc: 0.961412	valid_0's binary_logloss: 0.301245
[200]	valid_0's auc: 0.96195	valid_0's binary_logloss: 0

In [14]:
oof_lgbm = pd.DataFrame(oof_lgbm,columns=['oof_lgbmstack'])

In [15]:
oof_lgbm.to_csv('oof_lgbmstack.csv',index=False)

In [16]:
FOLDS = 5
SEED = 42
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
ix=0
params = {'objective': 'CrossEntropy', 'colsample_bylevel': 0.5, 'bootstrap_type': 'Bernoulli', 
 'max_depth': 7, 'l2_leaf_reg': 37, 'random_strength': 0.9, 'subsample': 0.9}
oof_catstack = np.zeros((len(oof_df)))
for train_ind, val_ind in skf.split(oof_df, oof_df.target):
    
    print(f"******* Fold {ix} ******* ")
    tr_x, val_x = (
        oof_df[oofCols].iloc[train_ind].reset_index(drop=True),
        oof_df[oofCols].iloc[val_ind].reset_index(drop=True),
    )
    tr_y, val_y = (
        oof_df.target.iloc[train_ind].reset_index(drop=True),
        oof_df.target.iloc[val_ind].reset_index(drop=True),
    )

    clf = CatBoostClassifier(**params, iterations= 15500,  random_state=SEED ,allow_writing_files=False)
    clf.fit(tr_x, tr_y, eval_set=[(val_x, val_y)], verbose=100,early_stopping_rounds=200)
    clf.save_model(f'cat_stacked_fold{ix}_seed{SEED}')
    clf.load_model(f'./cat_stacked_fold{ix}_seed{SEED}')  
    preds = clf.predict_proba(val_x)[:,1]
    acc = amex_metric_mod(val_y.values, preds)
    print('Kaggle Metric =',acc,'\n')
    
    # SAVE OOF
    oof_catstack[val_ind] = preds 
    
    del clf, tr_x, val_x, tr_y, val_y
    _ = gc.collect()
    
    ix = ix + 1
    
print('#'*25)
val_score = amex_metric_mod(oof_df.target, oof_catstack)
print(f"Amex metric: {val_score}")

******* Fold 0 ******* 
0:	learn: 0.6434994	test: 0.6438022	best: 0.6438022 (0)	total: 142ms	remaining: 36m 37s
100:	learn: 0.2138704	test: 0.2141190	best: 0.2141190 (100)	total: 8.49s	remaining: 21m 34s
200:	learn: 0.2128153	test: 0.2133126	best: 0.2133123 (198)	total: 17.7s	remaining: 22m 29s
300:	learn: 0.2123983	test: 0.2132278	best: 0.2132278 (300)	total: 25.7s	remaining: 21m 39s
400:	learn: 0.2119607	test: 0.2132245	best: 0.2132210 (397)	total: 34s	remaining: 21m 21s
500:	learn: 0.2113665	test: 0.2132558	best: 0.2132210 (397)	total: 42.4s	remaining: 21m 9s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.2132210319
bestIteration = 397

Shrink model to first 398 iterations.
Kaggle Metric = 0.7962680777709689 

******* Fold 1 ******* 
0:	learn: 0.6475566	test: 0.6476353	best: 0.6476353 (0)	total: 84ms	remaining: 21m 42s
100:	learn: 0.2138505	test: 0.2143378	best: 0.2143378 (100)	total: 8.03s	remaining: 20m 23s
200:	learn: 0.2127103	test: 0.2136572	best: 0.213657

In [17]:
oof_catstack = pd.DataFrame(oof_catstack,columns=['oof_catstack'])

In [18]:
oof_catstack.to_csv('oof_catstack.csv',index=False)

In [19]:
xgb_params = {
            'objective': 'binary:logitraw', 
            'tree_method': 'gpu_hist',
            'predictor':'gpu_predictor',
            'max_depth': 5,    
            'subsample':1,   
            'colsample_bytree': 0.65,  
            'gamma':5, 
            'min_child_weight':8,  
            'random_state':42,
            'eval_metric':'auc',
            'learning_rate':0.25,
            'num_parallel_tree':1,
    }

In [20]:
class IterLoadForDMatrix(xgb.core.DataIter):
    def __init__(self, df=None, features=None, target=None, batch_size=256*1024):
        self.features = features
        self.target = target
        self.df = df
        self.it = 0 # set iterator to 0
        self.batch_size = batch_size
        self.batches = int( np.ceil( len(df) / self.batch_size ) )
        super().__init__()

    def reset(self):
        '''Reset the iterator'''
        self.it = 0

    def next(self, input_data):
        '''Yield next batch of data.'''
        if self.it == self.batches:
            return 0 # Return 0 when there's no more batch.
        
        a = self.it * self.batch_size
        b = min( (self.it + 1) * self.batch_size, len(self.df) )
        dt = cudf.DataFrame(self.df.iloc[a:b])
        input_data(data=dt[self.features], label=dt[self.target]) #, weight=dt['weight'])
        self.it += 1
        return 1

In [21]:
oofCols = [col for col in oof_df.columns if 'oof' in col]

for col in oofCols:
    metric = amex_metric_mod(oof_df['target'], oof_df[col])
    
    print(f"{col} : {metric}")

oof_xgb : 0.7979803094946979
oof_automl : 0.7967220922562924
oof_lgbm : 0.7980939957511974
oof_xgbpy : 0.7974332770983097
oof_cat : 0.797516811438845
oof_lgbmquick : 0.7967421462859201
oof_cnn : 0.7938481749446653
oof_transfer1 : 0.7915815533640204
oof_transfer2 : 0.7918790490212578


In [22]:
#train = train.to_pandas() # free GPU memory
TRAIN_SUBSAMPLE = 1.0
gc.collect() 
FOLDS = 5

for SEED in [42]:
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
    oof = np.zeros((len(oof_df)))
    for fold,(train_idx, valid_idx) in enumerate(skf.split(
            oof_df, oof_df.target )):
    
    # TRAIN WITH SUBSAMPLE OF TRAIN FOLD DATA
        if TRAIN_SUBSAMPLE<1.0:
                np.random.seed(SEED)
                train_idx = np.random.choice(train_idx, 
                       int(len(train_idx)*TRAIN_SUBSAMPLE), replace=False)
                np.random.seed(None)
    
        print('#'*25)
        print('### SEED',SEED)
        print('### Fold',fold+1)
        print('### Train size',len(train_idx),'Valid size',len(valid_idx))
        print(f'### Training with {int(TRAIN_SUBSAMPLE*100)}% fold data...')
        print('#'*25)
    
        # TRAIN, VALID, TEST FOR FOLD K
        Xy_train = IterLoadForDMatrix(oof_df.loc[train_idx], oofCols, 'target')
        X_valid = oof_df.loc[valid_idx, oofCols]
        y_valid = oof_df.loc[valid_idx, 'target']
    

        dtrain = xgb.DeviceQuantileDMatrix(Xy_train, max_bin=511)
        dvalid = xgb.DMatrix(data=X_valid, label=y_valid)
    
         # TRAIN MODEL FOLD K
        model = xgb.train(xgb_params, 
                dtrain=dtrain,          
                evals=[(dtrain,'train'),(dvalid,'valid')],
                num_boost_round=9999,
                early_stopping_rounds=100,
                verbose_eval=100) 
        model.save_model(f'XGBstacked_fold{fold}.xgb')
            
        # INFER OOF FOLD K
        oof_preds = model.predict(dvalid)
        acc = amex_metric_mod(y_valid.values, oof_preds)
        print('Kaggle Metric =',acc,'\n')
    
        # SAVE OOF
        oof[valid_idx] = oof_preds
    
        del dtrain, Xy_train
        del X_valid, y_valid, dvalid, model
        _ = gc.collect()
    
print('#'*25)
acc = amex_metric_mod(oof_df.target, oof)
print('OVERALL CV Kaggle Metric =',acc)

#########################
### SEED 42
### Fold 1
### Train size 367130 Valid size 91783
### Training with 100% fold data...
#########################
[0]	train-auc:0.96191	valid-auc:0.96196
[100]	train-auc:0.96348	valid-auc:0.96320
[129]	train-auc:0.96348	valid-auc:0.96320
Kaggle Metric = 0.7967804843822882 

#########################
### SEED 42
### Fold 2
### Train size 367130 Valid size 91783
### Training with 100% fold data...
#########################
[0]	train-auc:0.96201	valid-auc:0.96127
[100]	train-auc:0.96354	valid-auc:0.96303
[124]	train-auc:0.96354	valid-auc:0.96303
Kaggle Metric = 0.7986731106403104 

#########################
### SEED 42
### Fold 3
### Train size 367130 Valid size 91783
### Training with 100% fold data...
#########################
[0]	train-auc:0.96194	valid-auc:0.96186
[100]	train-auc:0.96352	valid-auc:0.96311
[126]	train-auc:0.96352	valid-auc:0.96311
Kaggle Metric = 0.8002690299307352 

#########################
### SEED 42
### Fold 4
### Train size 367

In [23]:
oof = pd.DataFrame(oof,columns=['oof_xgbstack'])

In [24]:
oof.to_csv('oof_xgbstack.csv',index=False)

In [25]:
scaler = StandardScaler()
scaler = load(open('../input/amex9predscaler/predscaler','rb'))
oofCols = [col for col in oof_df.columns if 'oof' in col]
oof_df.loc[:,oofCols] =  scaler.transform(oof_df.loc[:,oofCols])

In [26]:
import os
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [27]:
class resnetModel(nn.Module):
    def __init__(self, num_features,hidden_size,ispretrain=False):
        super(resnetModel, self).__init__()
        self.ispretrain=ispretrain
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        
        self.batch_norm2 = nn.BatchNorm1d(num_features+hidden_size)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(num_features+hidden_size, hidden_size))
        self.batch_norm20 = nn.BatchNorm1d(hidden_size)
        self.dropout20 = nn.Dropout(0.5)
        self.dense20 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        

        self.batch_norm3 = nn.BatchNorm1d(2*hidden_size)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(2*hidden_size, hidden_size))
        self.batch_norm30 = nn.BatchNorm1d(hidden_size)
        self.dropout30 = nn.Dropout(0.5)
        self.dense30 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        

        self.batch_norm4 = nn.BatchNorm1d(2*hidden_size)
        self.dropout4 = nn.Dropout(0.5)
        if self.ispretrain:
          self.dense4 = nn.utils.weight_norm(nn.Linear(2*hidden_size, 1))
        else:
          self.dense5 = nn.utils.weight_norm(nn.Linear(2*hidden_size, 1))
    
    def forward(self, x):
        x1 = self.batch_norm1(x)
        x1 = F.elu(self.dense1(x1))
        x = torch.cat([x,x1],1)
        
        x2 = self.batch_norm2(x)
        x2 = self.dropout2(x2)
        x2 = F.elu(self.dense2(x2))
        x2 = self.batch_norm20(x2)
        x2 = self.dropout20(x2)
        x2 = F.elu(self.dense20(x2))
        x = torch.cat([x1,x2],1)

        x3 = self.batch_norm3(x)
        x3 = self.dropout3(x3)
        x3 = F.elu(self.dense3(x3))
        x3 = self.batch_norm30(x3)
        x3 = self.dropout30(x3)
        x3 = F.elu(self.dense30(x3))
        x3 = torch.cat([x2,x3],1)
        
        x3 = self.batch_norm4(x3)
        x3 = self.dropout4(x3)
        if self.ispretrain:
          x3 = self.dense4(x3)
        else:
          x3 = self.dense5(x3)
        return x3

In [28]:
skf = StratifiedKFold(n_splits=5)

for f,(t_idx,v_idx) in enumerate(skf.split(X=oof_df,y=oof_df.target)):
    oof_df.loc[v_idx,'kfold'] = int(f)

oof_df['kfold'] = oof_df['kfold'].astype(int)    

In [29]:
class AmexDataset:
    def __init__(self,features,target,noise=0.1):
        self.features = features
        self.target = target
        self.noise = noise
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self,idx):
        
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.target[idx] ,dtype=torch.float)          
        }
        return dct

In [30]:
def train_fn(model,optimizer,scheduler,loss_fn,dataloader,device):
    
    model.train()
    final_loss = 0
    
    for  data in dataloader:
        optimizer.zero_grad()
        inputs,target = data['x'].to(device),data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs[:,0],target.float())
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss

def valid_fn(model,loss_fn,dataloader,device):
    model.eval()
    final_loss = 0
    valid_preds = [] 
    
    for data in dataloader:
        inputs,target = data['x'].to(device),data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs[:,0],target.float())
        final_loss +=loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

In [31]:
class FineTuneScheduler:
    def __init__(self, epochs):
        self.epochs = epochs
        self.epochs_per_step = 0
        self.frozen_layers = []

    def copy_without_top(self, model, num_features):
        self.frozen_layers = []

        model_new = resnetModel(num_features,1024)
        model_new.load_state_dict(model.state_dict())

        # Freeze all weights
        for name, param in model_new.named_parameters():
            layer_index = name.split('.')[0][-1]

            if layer_index == 5:
                continue

            param.requires_grad = False

            # Save frozen layer names
            if layer_index not in self.frozen_layers:
                self.frozen_layers.append(layer_index)

        self.epochs_per_step = self.epochs // len(self.frozen_layers)
        
        hidden_size = 1024
        # Replace the top layers with another ones
        model_new.batch_norm4 = nn.BatchNorm1d(2*hidden_size)
        model_new.dropout4 = nn.Dropout(0.5)
        model_new.dense5 = nn.utils.weight_norm(nn.Linear(2*hidden_size, 1))
        model_new.to(DEVICE)
        return model_new

    def step(self, epoch, model):
        if len(self.frozen_layers) == 0:
            return

        if epoch % self.epochs_per_step == 0:
            last_frozen_index = self.frozen_layers[-1]
            
            # Unfreeze parameters of the last frozen layer
            for name, param in model.named_parameters():
                layer_index = name.split('.')[0][-1]

                if layer_index == last_frozen_index:
                    param.requires_grad = True

            del self.frozen_layers[-1]  # Remove the last layer as unfrozen

In [32]:
DEVICE = 'cuda'
EPOCHS = 15
BATCH_SIZE = 1024
LEARNING_RATE = 1e-3
NFOLDS = 5           #<-- Update
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

In [33]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1

        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
            
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [34]:
def run_training(fold,seed):
    seed_everything(seed)
    
    trn_idx = oof_df[oof_df['kfold'] != fold].index
    val_idx = oof_df[oof_df['kfold'] == fold].index
    
    train_df = oof_df[oof_df['kfold'] != fold].reset_index(drop=True)
    valid_df = oof_df[oof_df['kfold'] == fold].reset_index(drop=True)
    
    def train_model(model,fine_tune_scheduler=None):
    
        x_train,y_train = train_df[oofCols].values,train_df['target'].values
        x_valid,y_valid = valid_df[oofCols].values,valid_df['target'].values
        
      #  scaler = StandardScaler()
        
      #  x_train =  scaler.fit_transform(x_train)
      #  x_valid = scaler.transform(x_valid)
        
        train_dataset = AmexDataset(x_train,y_train)
        valid_dataset = AmexDataset(x_valid,y_valid)

        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        optimizer = torch.optim.Adam(model.parameters())
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing=0.001)

        oof = np.zeros((len(oof_df),1))
        best_loss = np.inf
        
        for epoch in range(EPOCHS):
            if fine_tune_scheduler is not None:
                fine_tune_scheduler.step(epoch, model)

            train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, DEVICE)
            valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
            print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss:.6f}, valid_loss: {valid_loss:.6f}")

            if np.isnan(valid_loss):
                break
            
            if valid_loss < best_loss:
                best_loss = valid_loss
                oof[val_idx] = valid_preds
                torch.save(model.state_dict(), f"MLP_stack_SEED{seed}_FOLD{fold}_.pth")
       
        return oof

    fine_tune_scheduler = FineTuneScheduler(EPOCHS)
    
    pretrained_model = resnetModel(len(oofCols),1024)
    pretrained_model.to(DEVICE)
    
    print('1st Stage')
    
    # Train on scored + nonscored targets
    train_model(pretrained_model)
    
    # Load the pretrained model with the best loss
    pretrained_model = resnetModel(len(oofCols),1024)
    pretrained_model.load_state_dict(torch.load(f"MLP_stack_SEED{seed}_FOLD{fold}_.pth"))
    pretrained_model.to(DEVICE)
                                     
    # Copy model without the top layer
    final_model = fine_tune_scheduler.copy_without_top(pretrained_model,len(oofCols))     
    
    print('2nd Stage / Fine Tuning....')
                                     
    oof = train_model(final_model,fine_tune_scheduler)       
                                    
    return oof                                 

In [35]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(oof_df),1))
    
    for fold in range(NFOLDS):
        oof_ = run_training(fold, seed)
        oof += oof_
    return oof

In [36]:
SEED = [42]  #<-- Update
oof = np.zeros((len(oof_df),1))

for seed in SEED:
    
    oof_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)

1st Stage
SEED: 42, FOLD: 0, EPOCH: 0, train_loss: 0.265219, valid_loss: 0.221917
SEED: 42, FOLD: 0, EPOCH: 1, train_loss: 0.219787, valid_loss: 0.215331
SEED: 42, FOLD: 0, EPOCH: 2, train_loss: 0.217899, valid_loss: 0.215224
SEED: 42, FOLD: 0, EPOCH: 3, train_loss: 0.217430, valid_loss: 0.213348
SEED: 42, FOLD: 0, EPOCH: 4, train_loss: 0.217137, valid_loss: 0.214621
SEED: 42, FOLD: 0, EPOCH: 5, train_loss: 0.216841, valid_loss: 0.213249
SEED: 42, FOLD: 0, EPOCH: 6, train_loss: 0.216904, valid_loss: 0.213605
SEED: 42, FOLD: 0, EPOCH: 7, train_loss: 0.216590, valid_loss: 0.213341
SEED: 42, FOLD: 0, EPOCH: 8, train_loss: 0.216676, valid_loss: 0.213450
SEED: 42, FOLD: 0, EPOCH: 9, train_loss: 0.216597, valid_loss: 0.213344
SEED: 42, FOLD: 0, EPOCH: 10, train_loss: 0.216319, valid_loss: 0.213782
SEED: 42, FOLD: 0, EPOCH: 11, train_loss: 0.216018, valid_loss: 0.213315
SEED: 42, FOLD: 0, EPOCH: 12, train_loss: 0.216265, valid_loss: 0.213313
SEED: 42, FOLD: 0, EPOCH: 13, train_loss: 0.216148,

In [37]:
my_ap = amex_metric_mod(oof_df.target,oof[:,0])
my_ap

0.7994078694513771

In [38]:
oof_mlpstack = pd.DataFrame(oof[:,0],columns=['oof_mlpstack'])

In [39]:
oof_mlpstack.to_csv('oof_mlpstack.csv',index=False)

## Manual Weights

In [40]:
# 11 models
# manual_preds = 0.0089*oof_xgb.oof_xgb + 0.1*oof_automl.oof_automl + 0.0078*oof_xgbpy.oof_xgbpy + 0.6*oof_cat.oof_cat + 0.39*oof_lgbm.oof_lgbm + 0.076*oof_lgbmquick.oof_lgbmquick + 0.2*oof_cnn.oof_cnn + 0.085*oof_transfer1.oof_transfer1 + 0.3*oof_transfer2.oof_transfer2 + 0.6*oof_lr.oof_lr

In [41]:
# amex_metric_mod(oof_df.target.values, manual_preds)